In [11]:
# from bs4 import BeautifulSoup 
# import requests
# ใช้ไม่ได้ เพราะองค์ประกอบจะเปลี่ยนไปทุกครั้งที่ Scrolling

import pandas as pd
from datetime import datetime
import time

from selenium import webdriver
from selenium.webdriver import ChromeOptions, Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

In [17]:
import json

with open("secret-key.json", "r") as file:
    key_json = json.load(file)
    
X_EMAIL = key_json["X_EMAIL"]
X_USERNAME = key_json["X_USERNAME"]
X_PASSWORD = key_json["X_PASSWORD"]

In [18]:
# X login automation by selenium
def login_X():
    
    options = ChromeOptions()
    options.add_argument("--start-maximized")
    options.add_experimental_option("excludeSwitches", ["enable-automation"])

    driver = webdriver.Chrome(options=options)
    url = "https://twitter.com/i/flow/login"
    driver.get(url)
    
    time.sleep(10)
    try:
        # กรอกชื่อผู้ใช้
        username = WebDriverWait(driver, 20).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, 'input[autocomplete="username"]'))
        )
        username.send_keys(X_USERNAME)
        username.send_keys(Keys.ENTER)

        # ตรวจสอบว่ามีให้กรอกอีเมลยืนยันหรือไม่
        time.sleep(3)  
        try:
            modal_header = driver.find_element(By.ID, "modal-header")
            if "Enter your phone number or email address" in modal_header.text:
                print("ระบบต้องการยืนยันด้วยอีเมลหรือหมายเลขโทรศัพท์")

                # กรอกอีเมล
                email_input = WebDriverWait(driver, 10).until(
                    EC.visibility_of_element_located((By.CSS_SELECTOR, 'input[data-testid="ocfEnterTextTextInput"]'))
                )
                email_input.send_keys(X_EMAIL)

                next_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[data-testid="ocfEnterTextNextButton"]'))
                )
                next_button.click()
        except Exception:
            pass

        # กรอกรหัสผ่าน
        password = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, 'input[name="password"]'))
        )
        password.send_keys(X_PASSWORD)
        password.send_keys(Keys.ENTER)
 
        time.sleep(10) # รอโหลดเข้าสู่ระบบ

    except Exception as e:
        print(f"Error during login: {e}")
        driver.quit()

    return driver

In [3]:
# กรองโฆษณาจาก X
def is_promotion(tweet_element):
    try:
        promo_element = tweet_element.find_element(By.XPATH, './/span[contains(text(), "ประชาสัมพันธ์")]')
        return True
    except:
        return False

In [4]:
# ดึง ID และ Username ของข้อความ
def extract_tweet_link(tweet_element):
    try:
       # หาองค์ประกอบ URL ของทวีต
        link_element = tweet_element.find_element(By.CSS_SELECTOR, 'a[href*="/status/"]')
        
        # ดึงค่า URL
        tweet_url = link_element.get_attribute("href")
        
        # ดึงค่า ID จาก URL (หลัง "/status/")
        tweet_id = tweet_url.split("/status/")[-1]
        
        # ดึงชื่อผู้โพสต์จาก URL (ก่อน "/status/")
        username = tweet_url.split("/status/")[0].split("/")[-1]
        
        return tweet_id, username
    except Exception as e:
        print(f"Error extracting tweet info: {e}")
        return None, None

# ID และ Username สามารถประกอบกันเป็น link เช่น https://x.com/malbiguerras/status/1860939499683074175

In [5]:
# ดึงข้อมูล จำนวนความเห็น จำนวนการรีโพสต์ จำนวนความชอบ จำนวนบุ๊คมาร์ก จำนวนการดู
def extract_tweet_metrics(tweet_element):
    try:
        metrics_element = tweet_element.find_element(By.XPATH, './/div[@role="group"]')
        metrics_text = metrics_element.get_attribute("aria-label")
        metrics = {
            "replies": 0,
            "retweets": 0,
            "likes": 0,
            "bookmarks": 0,
            "views": 0,
        }
        if metrics_text:
            # แยกข้อมูลออกจากกัน และแปลงตัวอักษร K M เป็นตัวเลข
            metrics_split = metrics_text.split(", ")
            for metric in metrics_split:
                if "การตอบกลับ" in metric:
                    metrics["replies"] = int(metric.split(" ")[0].replace("K", "000").replace("M", "000000"))
                elif "รีโพสต์" in metric:
                    metrics["retweets"] = int(metric.split(" ")[0].replace("K", "000").replace("M", "000000"))
                elif "ความชอบ" in metric:
                    metrics["likes"] = int(metric.split(" ")[0].replace("K", "000").replace("M", "000000"))
                elif "บุ๊คมาร์ก" in metric:
                    metrics["bookmarks"] = int(metric.split(" ")[0].replace("K", "000").replace("M", "000000"))
                elif "ดู" in metric:
                    metrics["views"] = int(metric.split(" ")[0].replace("K", "000").replace("M", "000000"))
        return metrics
    except:
        return None

In [6]:
# ดึงวันที่โพสต์จาก Tweet
def extract_tweet_date(tweet_element):
    try:
        # ค้นหาองค์ประกอบ time ใน tweet_element
        time_element = tweet_element.find_element(By.CSS_SELECTOR, 'time')
        
        # ดึงค่า datetime จาก attribute ของ time
        post_date_iso = time_element.get_attribute("datetime")
        
        if post_date_iso:
            # แปลง ISO 8601 เป็นรูปแบบ YYYY-MM-DD HH:MM:SS
            formatted_date = datetime.strptime(post_date_iso, "%Y-%m-%dT%H:%M:%S.%fZ").strftime("%Y-%m-%d %H:%M:%S")
            return formatted_date
            
    except Exception as e:
        print(f"Error extracting date: {e}")
    return None

In [7]:
# เลื่อนหน้าจอและดึงข้อมูล
def scroll_and_get_data(driver, raw_data):
    last_height = driver.execute_script("return document.body.scrollHeight")
    scroll_count = 0
    max_scrolls = 10  # จำนวนครั้งที่เลื่อนหน้าจอได้สูงสุด

    while scroll_count < max_scrolls:
        # ดึงโพสต์ทั้งหมดในหน้า
        tweets = driver.find_elements(By.CSS_SELECTOR, 'article')

        for tweet in tweets:
            if not is_promotion(tweet):  # กรองข้อความที่เป็นโฆษณาของ X
                try:
                    # ดึง ID และ Username
                    tweet_id, username = extract_tweet_link(tweet)
                    if tweet_id: 

                        # ดึงข้อความในโพสต์
                        text_element = tweet.find_element(By.CSS_SELECTOR, 'div[lang]')
                        text = text_element.text

                        # ดึงข้อมูลจำนวนต่าง ๆในโพสต์
                        metrics = extract_tweet_metrics(tweet)

                        # ดึงวันที่โพสต์  
                        tweet_date = extract_tweet_date(tweet)                        

                        if metrics and tweet_date:
                            # เก็บข้อมูลลงใน list
                            raw_data.append({
                                "ID": tweet_id,
                                "Tweet": text,
                                "Replies": metrics["replies"],
                                "Retweets": metrics["retweets"],
                                "Likes": metrics["likes"],
                                "Bookmarks": metrics["bookmarks"],
                                "Views": metrics["views"],
                                "Username": username,
                                "PostDate": tweet_date
                            })
                except Exception as e:
                    print(f"Error while scraping: {e}")

        # เลื่อนหน้าจอลง
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)  # รอโหลดเนื้อหาใหม่

        # ตรวจสอบว่ามีเนื้อหาใหม่หรือไม่
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            print("Reached the end of the page.")
            break
        last_height = new_height
        scroll_count += 1


In [8]:
# สร้าง data frame และ Update Timestamp
def process_data_frame(raw_data: pd.DataFrame, main_df : pd.DataFrame) -> pd.DataFrame:
    if not raw_data.empty:  # ตรวจสอบว่า raw_data(ข้อมูลใหม่) มีข้อมูล
        
        new_df = pd.DataFrame(raw_data)
        # เพิ่มคอลัมน์วันเวลาที่ดึงข้อมูล 
        new_df["Timestamp"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")  # วันที่และเวลาปัจจุบัน
        new_df["Timestamp"] = pd.to_datetime(new_df["Timestamp"])  

        # รวมข้อมูลใหม่เข้ากับ main_df
        combined_df = pd.concat([main_df, new_df], ignore_index=True)

    else:
        # ถ้าไม่มีข้อมูลใหม่ ให้คืนค่า main_df เดิม
        combined_df = main_df  

    # แสดง data frame
    print("แสดงข้อมูลหลังเพิ่ม Timestamp และรวมข้อมูลใหม่กับข้อมูลเก่า")
    with pd.option_context('display.max_colwidth', None):
        display(combined_df)

    return combined_df

In [9]:
# เช็ค id ซ้ำ
def check_id_duplicate(main_df):
    
    # แปลงเป็น datetime และดูที่เป็น null
    main_df['Timestamp'] = pd.to_datetime(main_df['Timestamp'], errors='coerce')
    
    # คัดกรอง ID ซ้ำ เลือกแถวที่มี Timestamp ล่าสุด
    main_df = main_df.sort_values(by="Timestamp").drop_duplicates(subset=["ID"], keep="last").reset_index(drop=True)
    
    # check main_df after drop ID
    print("drop ID duplicate")
    with pd.option_context('display.max_colwidth', None):
        display(main_df)

    return main_df

In [10]:
def web_scraping():
    
    driver = login_X()
    
    # เก็บ link ที่ต้องการทำ web scraping (link มี format ที่แน่นอน อาจเก็บแค่คีย์เวิร์ดแล้วนำไปแปลงเป็น link ในภายหลัง)
    url_list = {
        # 'มหาวิทยาลัยเกษตรศาสตร์':[
        #             "https://x.com/search?q=%23%E0%B9%80%E0%B8%81%E0%B8%A9%E0%B8%95%E0%B8%A3%E0%B8%A8%E0%B8%B2%E0%B8%AA%E0%B8%95%E0%B8%A3%E0%B9%8C&src=typed_query&f=top", # "#เกษตรศาสตร์(ยอดนิยม)"
        #             "https://x.com/search?q=%23%E0%B9%80%E0%B8%81%E0%B8%A9%E0%B8%95%E0%B8%A3%E0%B8%A8%E0%B8%B2%E0%B8%AA%E0%B8%95%E0%B8%A3%E0%B9%8C&src=recent_search_click&f=live" # "เกษตรศาสตร์(ล่าสุด)"
        #             ],
        'เน็ตทรู':[
                    "https://x.com/search?q=%23%E0%B9%80%E0%B8%99%E0%B9%87%E0%B8%95%E0%B8%97%E0%B8%A3%E0%B8%B9&src=typed_query&f=top" # "#เน็ตทรู(ยอดนิยม)"
                    ],
        'เน็ตเอไอเอส':[
                    "https://x.com/search?q=%23%E0%B9%80%E0%B8%99%E0%B9%87%E0%B8%95%E0%B9%80%E0%B8%AD%E0%B9%84%E0%B8%AD%E0%B9%80%E0%B8%AD%E0%B8%AA&src=typed_query&f=top" # "#เน็ตเอไอเอส(ยอดนิยม)"
                    ]
                }

    # read df เก่า
    column_types = {
        'ID': 'object',  
        'Tweet': 'object',  
        'Replies': 'int64', 
        'Retweets': 'int64', 
        'Bookmarks': 'int64', 
        'Views': 'int64', 
        'Username': 'object',
        'Category': 'object',
    }
    try:
        main_df = pd.read_csv('Tweet_data_scraping.csv', dtype=column_types, parse_dates=['Timestamp', 'PostDate'])
    except FileNotFoundError:
        main_df = pd.DataFrame(columns=["ID", "Tweet", "Replies", "Retweets", "Likes", "Bookmarks", "Views", "Username", "PostDate", "Timestamp", "Category"])  # ถ้าไม่มีไฟล์ สร้าง DataFrame ว่าง

    # วนลูปดึงข้อมูลใน url_list
    for key in url_list.keys():
        for url in url_list[key]:
            print(f"Scraping URL: {url}")
            
            # ข้อมูลใหม่
            raw_data = [] 
            
            # ไปที่ URL แฮชแท็ก
            driver.get(url)  
            time.sleep(5) 

            scroll_and_get_data(driver, raw_data)  # ดึงข้อมูลจากหน้าเว็บ

            # สร้าง DataFrame ใหม่จาก raw_data เพื่อเพิ่มคอลัมน์หมวดหมู่ตาม key
            temp_df = pd.DataFrame(raw_data)
            temp_df["Category"] = key
            
            print("แสดงข้อมูลหลังเพิ่ม Category")
            with pd.option_context('display.max_colwidth', None):
                display(temp_df)
            

            main_df = process_data_frame(temp_df, main_df) # รวมกับ df เก่า
    
    main_df = check_id_duplicate(main_df) # drop id 

    # ปิดเบราว์เซอร์
    driver.quit() 

    return main_df, raw_data

In [11]:
def save_csv(main_df):

    file_path = "Tweet_data_scraping.csv"
    
    column_types = {
            'ID': 'object',  
            'Tweet': 'object',  
            'Replies': 'int64', 
            'Retweets': 'int64', 
            'Bookmarks': 'int64', 
            'Views': 'int64', 
            'Username': 'object',
            'PostDate' : 'datetime64[ns]',
            'Timestamp' : 'datetime64[ns]',
            'Category': 'object',
        }
    
    # แปลง Data type
    for column, dtype in column_types.items():
        main_df[column] = main_df[column].astype(dtype)

    # บันทึกข้อมูลลงไฟล์ CSV
    main_df.to_csv(
        file_path,
        index=False,
        encoding="utf-8",
        # dtype=column_types
    )

Web scraping

In [12]:
main_df, raw_data = web_scraping()

Scraping URL: https://x.com/search?q=%23%E0%B9%80%E0%B8%81%E0%B8%A9%E0%B8%95%E0%B8%A3%E0%B8%A8%E0%B8%B2%E0%B8%AA%E0%B8%95%E0%B8%A3%E0%B9%8C&src=typed_query&f=top
Reached the end of the page.
แสดงข้อมูลหลังเพิ่ม Category


,ID,Tweet,Replies,Retweets,Likes,Bookmarks,Views,Username,PostDate,Category
0,1865962879889092684,"ฝากถึง ว่าที่ KU 85 \nพี่ ""เฮ"" วชิรวิทย์ ฮอ นิสิตKU 84 คณะเศรษฐศาสตร์ และนักแสดงซีรีส์\nชวนน้อง ๆ นักเรียนมัธยมปลาย เข้าร่วมงานนิทรรศการตลาดนัดหลักสูตรอุดมศึกษา\nวันที่ 19-20 ธันวาคม\nเตรียมตัวกันให้พร้อม บอกเลยว่าปีนี้เราจัดเต็ม\n#kasetsart #ku #tcas #tcas68 #ทีมมก #เกษตรศาสตร์",0,1,6,0,1440,kasetsart_ku,2024-12-09 03:33:13,มหาวิทยาลัยเกษตรศาสตร์
1,1865238169463198016,เตรียมพบกับ KU Open House 2024 \n\n 19-20 ธ.ค. 67\nอาคารจักรพันธ์เพ็ญศิริ ม.เกษตร\n\nเปิดโอกาสใหม่ในโลกการเรียนรู้กับ KU!\n มาลงทะเบียนกัน : https://shorturl.asia/vIFwk\n\n#KUopenhouse2024 #เลือกแนวทางวางอนาคต2567 \n#kasetsart #เกษตรศาสตร์ #ทีมเกษตร #tcas68,0,6,12,3,2029,kasetsart_ku,2024-12-07 03:33:28,มหาวิทยาลัยเกษตรศาสตร์
2,1869281208045301989,มหกรรมความสนุกที่\nใหญ่ที่สุดกลับมาแล้ว\nOpen House KU 2024 ที่จะทำให้คุณ\nอยากมาเป็นครอบครัวKU\nคณะไหนมีกิจกรรมอะไร วันไหนบ้าง เซฟ\nข้อมูลเตรียมตัวกันได้เลย\nเวิร์กชอปสุดปัง เจอกับพี่ๆทุกคณะ\nพร้อมตอบทุกคำถาม!!\n19-20 ธ.ค. 67\n#เกษตรศาสตร์ #ทีมมก #kasetsart #ku,0,3,17,6,2143,kasetsart_ku,2024-12-18 07:19:04,มหาวิทยาลัยเกษตรศาสตร์
3,1737249737257173400,ขอเชิญน้อง ๆ มัธยมศึกษาเข้าร่วมงาน\nนิทรรศการตลาดนัดหลักสูตรอุดมศึกษา\nโครงการเลือกแนวทางวางอนาคต\nที่ ม.เกษตรศาสตร์ กับหลักสูตร วท.บ. สาขาอาหาร โภชนาการ และการกำหนดอาหาร\n#FNDKU #KU #kuopenhouse2023 #เกษตรศาสตร์ #TCAS67 #TCAS68 #นักโภชนาการ #นักกําหนดอาหาร #dietitian #Dietetics,0,2,2,0,958,noinadekdee,2023-12-19 23:13:08,มหาวิทยาลัยเกษตรศาสตร์
4,1665622712612532224,สินค้ามือสอง ราคาปุกปิกก~~ \nเรากำลังจะย้ายออกจากหอแถว ม.เกษตร บางเขน สิ้นเดือนนี้ เลยขอมาเปิดแผงขายของ \n#dek66 #เกษตรศาสตร์,2,4,1,1,1335,panangpangtod,2023-06-05 07:32:55,มหาวิทยาลัยเกษตรศาสตร์
5,1859963408462029205,DM #กทม #thetraffic #เกษตรศาสตร์ #บางเขน,0,0,5,0,1144,Dayly47449895,2024-11-22 14:13:27,มหาวิทยาลัยเกษตรศาสตร์
6,1839602307543228626,#โตโน่ ภาคิน คำวิลัยศักดิ์ ขึ้นแท่นรักษาการประธานสโมสร #เกษตรศาสตร์เอฟซี ในศึกไทยลีก 2 หลังเกิดปัญหาการบริหารภายในทีมอย่างหนัก ตั้งเป้าสร้างทีมอย่างยั่งยืน พร้อมชวนแฟนบอลอุดหนุนเสื้อแข่ง เพื่อหาเงินมาหมุนเวียนในสโมสร\n\n#ภาคินคำวิลัยศักดิ์ #โตโน่ภาคิน #ไทยลีก2 #เกษตรศาสตร์,3,16,124,2,51333,Thairath_News,2024-09-27 09:45:42,มหาวิทยาลัยเกษตรศาสตร์
7,1854206878894658037,ใครยังไม่นอนทักมทคุยได้นะคะเหงา!!! มาดูของดีที่ห้องเราไหม #สาวสอง #สาวอวบ #โสด #เรียลชาย #รัชดา36 #รัชโยธิน #ลาดพร้าว #เสนานิคม #เกษตรศาสตร์ #บางเขน #ทักมาคุยกัน #รัชดา #กทมนัด #กรุงเทพนัด #fwbกทม,0,2,6,0,2428,MAPING21228,2024-11-06 16:59:04,มหาวิทยาลัยเกษตรศาสตร์
8,1833032078444646683,ใครทีม #เกษตร ยกมือขึ้น แล้วเตรียมหาตี้ \nเพราะ ม.เกษตร บางเขน กำลังจะมี Open House!\nในวันที่ 19 - 20 ธ.ค. 67 นี้ งานฟรี ไม่มีค่าใช้จ่าย \n.\n#TCASter #Mytcas #TCAS68 #DEK68 #KU #KasetsartUniversity #KUOpenHouse #OpenHouse2024 #เกษตรศาสตร์ #มหาวิทยาลัยเกษตรศาสตร์ #เกษตรบางเขน,0,4,17,3,2954,TCASterApp,2024-09-09 06:37:58,มหาวิทยาลัยเกษตรศาสตร์
9,1746373013745066119,ร้านเปิดนะครับ สั่งได้ตลอดพร้อมส่งเลยครับ เปิดร้านทุกวัน ทักมาสอบถามก่อนได้ครับ #ฟาเทค #ยาน้ำ #เขียวเหลืองราคาส่ง #ห้วยขวาง #ลาดพร้าว #รัชดา #ดินแดง #พระราม9 #จรัญ #ปทุมวัน #รัชโยธิน #พหลโยธิน #อนุสาวรีย์ #เกษตรศาสตร์ #ประชาสงเคราะห์ #อโศก #ลาดปลาเค้า,0,1,2,0,2028,palaployy0404,2024-01-14 03:25:47,มหาวิทยาลัยเกษตรศาสตร์


แสดงข้อมูลหลังเพิ่ม Timestamp และรวมข้อมูลใหม่กับข้อมูลเก่า


,ID,Tweet,Replies,Retweets,Likes,Bookmarks,Views,Username,PostDate,Timestamp,Category
0,1783681490318143902,รศ. ดร.วีระภาส คุณรัตนสิริ รองคณบดีฝ่ายสารสนเทศ บัณฑิตวิทยาลัย กล่าวเปิดการอบรมเชิงปฏิบัติการ การใช้งาน iThesis ออนไลน์ ผ่านโปรแกรม Cisco WebEx ให้แก่นิสิตระดับบัณฑิตศึกษาของ มก. เมื่อวันที่ 24 เม.ย. 2567\n\n#เกษตรศาสตร์ #บัณฑิตวิทยาลัย #Kasetsart #KU #GradKU #iThesis #KUiThesis,0,0,0,0,296,Grad_KU,2024-04-26 02:16:21,2024-11-29 01:24:19,เกษตรศาสตร์
1,1797140769465372679,“ม.เกษตร” ต่อยอดหลักสูตร “KU CARE” รุ่น2 ยกระดับผู้บริหารระดับสูงขององค์กร สู่มาตรฐานอย่างยั่งยืน https://energynewscenter.com/%e0%b8%a1-%e0%b9%80%e0%b8%81%e0%b8%a9%e0%b8%95%e0%b8%a3-%e0%b8%95%e0%b9%88%e0%b8%ad%e0%b8%a2%e0%b8%ad%e0%b8%94%e0%b8%ab%e0%b8%a5%e0%b8%b1%e0%b8%81%e0%b8%aa%e0%b8%b9%e0%b8%95%e0%b8%a3-ku-care/… ผ่าน \n@EnergyNewsCent1\n#KU Care #เกษตรศาสตร์ #เคยู แคร์ #นโยบายสีเขียว #ธุรกิจ #ความยั่งยืน #เน็ต ซีโร่,0,0,1,0,173,EnergyNewsCent1,2024-06-02 05:38:43,2024-11-29 01:24:19,เกษตรศาสตร์
2,1725822429036175474,เกิดเหตุเพลิงไหม้ ใน ม.เกษตรศาสตร์ บางเขน ระดมกำลังดับทัน รอตรวจสาเหตุต่อ\n\nhttps://komchadluek.net/news/crime/563628…\n\n#ไฟไหม้ #เกษตรศาสตร์ #บางเขน #ข่าว #คมชัดลึก,0,0,2,0,2128,Kom_chad_luek,2023-11-18 10:25:05,2024-11-29 01:24:19,เกษตรศาสตร์
3,1710271184476987559,วิศวะอินเตอร์ #เกษตรศาสตร์ หลักสูตร IUP vs IDDP เรียนวิศวะเหมือนกัน แต่หลักสูตรเป็นวิศวะกันคนละสายเลยจ้า\n\nสมัครติว http://bit.ly/3AVZOt6\n\n#เรียนGED #เรียนไอเอลที่ไหนดี #สอบเทียบged #ทีมเกษตรศาสตร์ #ทีมมก #ทีมKU #KU #มก #DEK67 #DEK68 #TCAS #TCAS67 #ติวGEDที่ไหนดี #ADMISSION,0,1,1,0,1098,theplannertutor,2023-10-06 12:30:00,2024-11-29 01:24:19,เกษตรศาสตร์
4,1737125751772864562,พบกับหลักสุูตร วท.บ. สาขาอาหาร โภชนาการ และการกําหนดอาหาร (บูธ คณะเกษตร A11-14) ที่งาน KU Open House 2023 มหาวิทยาลัยเกษตรศาสตร์ บางเขน วันที่ 21-22 ธันวาคมนี้\n\n#FNDKU #AGGIEKU #KU #kuopenhouse2023 #เกษตรศาสตร์ #เกษตรบางเขน\n#TCAS67 #TCAS68 #นักโภชนาการ #นักกําหนดอาหาร #dietitian,0,0,0,0,1078,piiisut,2023-12-19 15:00:28,2024-11-29 01:24:19,เกษตรศาสตร์
...,...,...,...,...,...,...,...,...,...,...,...
416,1797140769465372679,“ม.เกษตร” ต่อยอดหลักสูตร “KU CARE” รุ่น2 ยกระดับผู้บริหารระดับสูงขององค์กร สู่มาตรฐานอย่างยั่งยืน https://energynewscenter.com/%e0%b8%a1-%e0%b9%80%e0%b8%81%e0%b8%a9%e0%b8%95%e0%b8%a3-%e0%b8%95%e0%b9%88%e0%b8%ad%e0%b8%a2%e0%b8%ad%e0%b8%94%e0%b8%ab%e0%b8%a5%e0%b8%b1%e0%b8%81%e0%b8%aa%e0%b8%b9%e0%b8%95%e0%b8%a3-ku-care/… ผ่าน \n@EnergyNewsCent1\n#KU Care #เกษตรศาสตร์ #เคยู แคร์ #นโยบายสีเขียว #ธุรกิจ #ความยั่งยืน #เน็ต ซีโร่,0,0,1,0,174,EnergyNewsCent1,2024-06-02 05:38:43,2024-12-24 04:43:33,มหาวิทยาลัยเกษตรศาสตร์
417,1753447682738450548,วันนี้ 2 กุมภาพันธ์ #ครบรอบ81ปี #เกษตรศาสตร์ \nข้าพระพุทธเจ้า ในนามนิสิตเก่า KU\nน้อมรำลึกในพระมหากรุณาธิคุณ \n-----------\n2 กุมภาพันธ์ พ.ศ. 2486 \nสถาปนามหาวิทยาลัยเกษตรศาสตร์,0,1,2,0,255,NightSun1911,2024-02-02 15:57:59,2024-12-24 04:43:33,มหาวิทยาลัยเกษตรศาสตร์
418,1813966182757921110,เชิญรับชมรับฟังและร่วมการเสวนาทางวิชาการ เรื่อง จบปัญหาการรุกรานแหล่งน้ำไทย จากปลาหมอคางดำ 19 ก.ค.67 เวลา 9.30 – 12.00 น.ผ่านเครือข่ายวิทยุ ม.ก. 4 ภูมิภาค อ่านต่อ\nhttps://facebook.com/share/p/Zn2LLUgzKaVQANvn/?mibextid=oFDknk…\n#กรมประมง #ปลาหมอสีคางดำ #วิทยุมก #เกษตรศาสตร์ #เกษตรกลางบางเขน #kuradionetwork,0,1,2,0,296,KuSongkhla,2024-07-18 15:56:54,2024-12-24 04:43:33,มหาวิทยาลัยเกษตรศาสตร์
419,1729817479826862501,#รอบพอร์ต โค้งสุดท้าย #dek67 ที่อยากติดรอบนี้ รีบปรึกษารุ่นพี่ตัวจริงมีประสบการณ์ตรงติดรอบ Portfolio มาแล้ว จองคิวโค้ชได้ที่ https://bit.ly/3R3PRmX\n.\nสอบถามเพิ่มเติม Line \n@coachdd\n@coachdd\n #COACHDD #TCAS #เตรียมสอบ #tcas67 #รอบportfolio #KMITL #KMUTT #มศว #เกษตรศาสตร์ #ศิลปากร,0,2,2,1,2572,coachddbydekd,2023-11-29 11:00:00,2024-12-24 04:43:33,มหาวิทยาลัยเกษตรศาสตร์


Scraping URL: https://x.com/search?q=%23%E0%B9%80%E0%B8%81%E0%B8%A9%E0%B8%95%E0%B8%A3%E0%B8%A8%E0%B8%B2%E0%B8%AA%E0%B8%95%E0%B8%A3%E0%B9%8C&src=recent_search_click&f=live
แสดงข้อมูลหลังเพิ่ม Category


...

,ID,Tweet,Replies,Retweets,Likes,Bookmarks,Views,Username,PostDate,Category
0,1796478907799273938,ทำไมเน็ตไฟเบอร์เดือนนี้ถูกจังคะ เกิดอะไรขึ้นเพราะเราย้ายมาต่างจังหวัด อันนี้สมัครให้คนที่บ้าน คนที่บ้านก็บอกล่มไปอาทิตย์นึง แต่ก็ไม่คิดว่าจะถูกขนาดนี้ หรืออะไรยังไง #เน็ตเอไอเอส #ais #เน็ตล่ม #เน็ตaisล่ม,0,0,0,0,3980,pichayathida_a,2024-05-31 09:48:43,เน็ตเอไอเอส
1,1847069253775904905,แกเปนไรอะ #เน็ตเอไอเอส เราจะดูช้างมันดูไม่ได้เนี่ย!!! #ais,1,0,0,0,212,ilovemoney1313,2024-10-18 00:16:41,เน็ตเอไอเอส
2,1862170683675206017,ขึ้น 5G ด้วยนะ นี่ขนาดแค่จะเช็คความเร็วแต่หมุนไม่หยุด สภาพค่าเน็ตเดือนละพัน #เน็ตค่ายเขียว #เน็ตเอไอเอส #เน็ตais,1,0,1,0,1109,deepfriedjpfood,2024-11-28 16:24:23,เน็ตเอไอเอส
3,1828466577462956124,สมัครแพ็กอีกอย่าง ได้อีกอย่าง หลอกลวงผู้บริโภค #net #เน็ตaisซิมใหม่ #เน็ตaisไม่ลดสปีด #เน็ตAIS #เน็ตวันทูคอล #AIS5G #เน็ตไม่อั้นไม่ลดสปีด #เน็ตไม่อั้น #โปรเน็ตais #เน็ตฟลิกซ์ #เน็ตเอไอเอส #เน็ตมือถือ #เน็ตais #ais #Ais #เน็ตไม่ลดสปีด #วันทูคอล #AISFibre #AISPLAY #เน็ตฟลิกซ์ไทย,0,0,0,0,1687,chi26429,2024-08-27 16:16:17,เน็ตเอไอเอส
4,1796692105890549831,เน็ตไม่อั้นไม่ลดสปีด ปี 2567 เน็ตAISยอดนิยม #net #เน็ตaisซิมใหม่ #เน็ตaisไม่ลดสปีด #เน็ตAIS #เน็ตวันทูคอล #AIS5G #เน็ตไม่อั้นไม่ลดสปีด #เน็ตไม่อั้น #โปรเน็ตais #เน็ตฟลิกซ์ #เน็ตเอไอเอส #เน็ตมือถือ #เน็ตais #ais #Ais #เน็ตไม่ลดสปีด #วันทูคอล #AISFibre #AISPLAY #เน็ตฟลิกซ์ไทย,0,3,2,4,2887,netais_,2024-05-31 23:55:53,เน็ตเอไอเอส
5,1719187625897230718,เน็ตไม่อั้นไม่ลดสปีด ปี 2566 \n เน็ตเอไอเอส-วันทูคอลยอดนิยม\n สมัครได้ทุกซิม \nAIS-12call #net #เน็ตaisซิมใหม่ #เน็ตaisไม่ลดสปีด #เน็ตAIS #เน็ตวันทูคอล #AIS5G #เน็ตไม่อั้นไม่ลดสปีด #เน็ตไม่อั้น #เน็ตบ้าน #เน็ตฟลิกซ์ #เน็ตเอไอเอส #เน็ตมือถือ #เน็ตais #ais #Ais #เน็ตไม่ลดสปีด,0,0,1,0,893,TANTAWAN014849,2023-10-31 03:00:45,เน็ตเอไอเอส
6,1831318570464428079,#เน็ตais #เน็ตเอไอเอส\n\nช่วงนี้เน็ตผีมากกก ขึ้น 3g ไม่พอ โหลดไม่ได่ ใช้กูเกิ้ลแมพยังไมาได่เลยค่ะ โทรไปหา call center คือบอกว่าให้จำมาว่าตรงไหนไม่มีสัญญาณบ้าง ใล้ลูกค้าไปตรวจสอบซิมการ์ด ไปตรวจสอบเครื่องตัวเองว่ามีปัญหาเรื่องสํญญาณมั้ย จะบ้าตาย (1),2,1,1,0,1047,littlesweetp3,2024-09-04 13:09:06,เน็ตเอไอเอส
7,1788863086470001102,ทำไม #เน็ตเอไอเอส #ais ลงเหลือ 3G ?,4,6,11,0,3892,mondaymay_,2024-05-10 09:26:09,เน็ตเอไอเอส
8,1788871940293423165,ขนาดGOMOเครือเดียวกับaisก็มิรอด #เน็ตเอไอเอส,0,0,0,0,282,nnbuxngginee,2024-05-10 10:01:20,เน็ตเอไอเอส
9,1640602403954368512,สมัครเน็ตบ้าน เร็ว แรง 399 บาท/เดือน \n\n#โปรเน็ตAis #เน็ตเอไอเอส,0,0,2,0,2445,12callnetpro,2023-03-28 06:31:08,เน็ตเอไอเอส


แสดงข้อมูลหลังเพิ่ม Timestamp และรวมข้อมูลใหม่กับข้อมูลเก่า


,ID,Tweet,Replies,Retweets,Likes,Bookmarks,Views,Username,PostDate,Timestamp,Category
0,1783681490318143902,รศ. ดร.วีระภาส คุณรัตนสิริ รองคณบดีฝ่ายสารสนเทศ บัณฑิตวิทยาลัย กล่าวเปิดการอบรมเชิงปฏิบัติการ การใช้งาน iThesis ออนไลน์ ผ่านโปรแกรม Cisco WebEx ให้แก่นิสิตระดับบัณฑิตศึกษาของ มก. เมื่อวันที่ 24 เม.ย. 2567\n\n#เกษตรศาสตร์ #บัณฑิตวิทยาลัย #Kasetsart #KU #GradKU #iThesis #KUiThesis,0,0,0,0,296,Grad_KU,2024-04-26 02:16:21,2024-11-29 01:24:19,เกษตรศาสตร์
1,1797140769465372679,“ม.เกษตร” ต่อยอดหลักสูตร “KU CARE” รุ่น2 ยกระดับผู้บริหารระดับสูงขององค์กร สู่มาตรฐานอย่างยั่งยืน https://energynewscenter.com/%e0%b8%a1-%e0%b9%80%e0%b8%81%e0%b8%a9%e0%b8%95%e0%b8%a3-%e0%b8%95%e0%b9%88%e0%b8%ad%e0%b8%a2%e0%b8%ad%e0%b8%94%e0%b8%ab%e0%b8%a5%e0%b8%b1%e0%b8%81%e0%b8%aa%e0%b8%b9%e0%b8%95%e0%b8%a3-ku-care/… ผ่าน \n@EnergyNewsCent1\n#KU Care #เกษตรศาสตร์ #เคยู แคร์ #นโยบายสีเขียว #ธุรกิจ #ความยั่งยืน #เน็ต ซีโร่,0,0,1,0,173,EnergyNewsCent1,2024-06-02 05:38:43,2024-11-29 01:24:19,เกษตรศาสตร์
2,1725822429036175474,เกิดเหตุเพลิงไหม้ ใน ม.เกษตรศาสตร์ บางเขน ระดมกำลังดับทัน รอตรวจสาเหตุต่อ\n\nhttps://komchadluek.net/news/crime/563628…\n\n#ไฟไหม้ #เกษตรศาสตร์ #บางเขน #ข่าว #คมชัดลึก,0,0,2,0,2128,Kom_chad_luek,2023-11-18 10:25:05,2024-11-29 01:24:19,เกษตรศาสตร์
3,1710271184476987559,วิศวะอินเตอร์ #เกษตรศาสตร์ หลักสูตร IUP vs IDDP เรียนวิศวะเหมือนกัน แต่หลักสูตรเป็นวิศวะกันคนละสายเลยจ้า\n\nสมัครติว http://bit.ly/3AVZOt6\n\n#เรียนGED #เรียนไอเอลที่ไหนดี #สอบเทียบged #ทีมเกษตรศาสตร์ #ทีมมก #ทีมKU #KU #มก #DEK67 #DEK68 #TCAS #TCAS67 #ติวGEDที่ไหนดี #ADMISSION,0,1,1,0,1098,theplannertutor,2023-10-06 12:30:00,2024-11-29 01:24:19,เกษตรศาสตร์
4,1737125751772864562,พบกับหลักสุูตร วท.บ. สาขาอาหาร โภชนาการ และการกําหนดอาหาร (บูธ คณะเกษตร A11-14) ที่งาน KU Open House 2023 มหาวิทยาลัยเกษตรศาสตร์ บางเขน วันที่ 21-22 ธันวาคมนี้\n\n#FNDKU #AGGIEKU #KU #kuopenhouse2023 #เกษตรศาสตร์ #เกษตรบางเขน\n#TCAS67 #TCAS68 #นักโภชนาการ #นักกําหนดอาหาร #dietitian,0,0,0,0,1078,piiisut,2023-12-19 15:00:28,2024-11-29 01:24:19,เกษตรศาสตร์
...,...,...,...,...,...,...,...,...,...,...,...
649,1811024706520371690,เน็ตไม่อั้นไม่ลดสปีด ปี 2567 เน็ตAISยอดนิยม #net #เน็ตaisซิมใหม่ #เน็ตaisไม่ลดสปีด #เน็ตAIS #เน็ตวันทูคอล #AIS5G #เน็ตไม่อั้นไม่ลดสปีด #เน็ตไม่อั้น #โปรเน็ตais #LISA #เน็ตเอไอเอส #เน็ตมือถือ #เน็ตais #ais #Ais #เน็ตไม่ลดสปีด #วันทูคอล #AISFibre #AISPLAY #Samsung,0,1,1,1,209,TANTAWAN014849,2024-07-10 13:08:31,2024-12-24 04:46:36,เน็ตเอไอเอส
650,1670450354583126017,เน็ตไม่อั้นไม่ลดสปีด ปี 2566 เน็ตAISยอดนิยม #net #เน็ตaisซิมใหม่ #เน็ตaisไม่ลดสปีด #เน็ตAIS #เน็ตวันทูคอล #AIS5G #เน็ตไม่อั้นไม่ลดสปีด #เน็ตไม่อั้น #โปรเน็ตais #เน็ตฟลิกซ์ #เน็ตเอไอเอส #เน็ตมือถือ #เน็ตais #ais #Ais #เน็ตไม่ลดสปีด #วันทูคอล #AISFibre #AISPLAY #เน็ตฟลิกซ์ไทย,0,1,5,3,3401,PhawiP,2023-06-18 15:16:14,2024-12-24 04:46:36,เน็ตเอไอเอส
651,1682206858315194369,โปรเน็ต 𝐀𝐈𝐒 ไม่ลดสปีด \n#net #เน็ตaisซิมใหม่ #เน็ตaisไม่ลดสปีด #เน็ตวันทูคอล #AIS5G #เน็ตไม่อั้นไม่ลดสปีด #เน็ตไม่อั้น #เน็ตฟลิกซ์ #เน็ตเอไอเอส #เน็ตมือถือ #เน็ตais #ais #Ais #เน็ตไม่ลดสปีด #วันทูคอล #เน็ตฟลิกซ์ไทย #โปรเน็ตais #สมัครเน็ต #แพ็กเน็ตais #เน็ตเสริมais,0,2,4,0,1837,netais_,2023-07-21 01:52:23,2024-12-24 04:46:36,เน็ตเอไอเอส
652,1680381832918093825,เน็ตais ไม่อั้นไม่ลดสปีด ปี 2566 #net #เน็ตaisซิมใหม่ #เน็ตaisไม่ลดสปีด #เน็ตAIS #เน็ตวันทูคอล #AIS5G #เน็ตไม่อั้นไม่ลดสปีด #เน็ตไม่อั้น #โปรเน็ตais #เน็ตฟลิกซ์ #เน็ตเอไอเอส #เน็ตมือถือ #เน็ตais #ais #Ais #เน็ตไม่ลดสปีด #วันทูคอล #AISFibre #AISPLAY #เน็ตฟลิกซ์ไทย,0,1,4,0,2576,netais_,2023-07-16 01:00:23,2024-12-24 04:46:36,เน็ตเอไอเอส


droup ID duplicate


,ID,Tweet,Replies,Retweets,Likes,Bookmarks,Views,Username,PostDate,Timestamp,Category
0,1783681490318143902,รศ. ดร.วีระภาส คุณรัตนสิริ รองคณบดีฝ่ายสารสนเทศ บัณฑิตวิทยาลัย กล่าวเปิดการอบรมเชิงปฏิบัติการ การใช้งาน iThesis ออนไลน์ ผ่านโปรแกรม Cisco WebEx ให้แก่นิสิตระดับบัณฑิตศึกษาของ มก. เมื่อวันที่ 24 เม.ย. 2567\n\n#เกษตรศาสตร์ #บัณฑิตวิทยาลัย #Kasetsart #KU #GradKU #iThesis #KUiThesis,0,0,0,0,296,Grad_KU,2024-04-26 02:16:21,2024-11-29 01:24:19,เกษตรศาสตร์
1,1728921798811746633,มาวิ่งงาน Run For Fisheries #RunForFisheries2023 ที่ ม.เกษตรศาสตร์ #ku #เกษตรศาสตร์ มาวิ่งที่นี่เป็นครั้งแรก ... ลงรุ่น 5KM แบบขำๆไม่ซีเรียส แต่เหนื่อยเอาเรื่อง ปกติจะมาที่นี่เพราะงาน Barcamp,0,0,0,0,193,wissarut106,2023-11-26 23:40:53,2024-11-29 01:24:19,เกษตรศาสตร์
2,1657651670560698368,ถ้าเราอยากอยู่ห้องเดียวกับเพื่อนสามารถทำยังไงได้มั้งคะ พอดีทักไปถามม.แล้วเค้าบอกสุ่มอย่างเดียวเลยค่ะ แนะนำได้นะคะ #หอในนอก #ทีมมก #เกษตรบางเขน #เกษตรศาสตร์ ปี1ค่ะ,1,0,0,0,450,bambysocute,2023-05-14 07:38:50,2024-11-29 01:24:19,เกษตรศาสตร์
3,1691777404833009925,แอดไลน์ http://bit.ly/3AVZOt6 แอดไลน์ตอนนี้ กดรับฟรีทันที!! “Universities Guide Book” ที่รวบรวม Requirement หลักสูตรอินเตอร์ขวัญใจเด็ก GEN Z เอาไว้\n\n#DEK67 #TCAS #จุฬา #มหิดล #มข #มช #เกษตรศาสตร์ #tcas67 #ธรรมศาสตร์ #MUIC #ศิลปากร,0,1,0,0,917,theplannertutor,2023-08-16 11:42:19,2024-11-29 01:24:19,เกษตรศาสตร์
4,1660348104267423744,รบกวนช่วยรีหน่อยค่ะ\nเวลาประมาณ 23.40 น. ใครที่ขับรถผ่านบริเวณสะพานข้ามแยกเกษตรฝั่งขาออก (ฝั่งม.เกษตร) แล้วพบเห็นอุบัติเหตุรถปิ๊กอัพชนรถจักรยานยนต์ รบกวนติดต่อเรามาหน่อยได้มั้ยคะ พอดีผู้ประสบเหตุเป็นพี่ชายเรา คนขับชนแล้วหนีค่ะ #อุบัติเหตุ #รถชน #เกษตรบางเขน #เกษตรศาสตร์,1,38,2,0,2960,DZemb_1991,2023-05-21 18:13:30,2024-11-29 01:24:19,เกษตรศาสตร์
...,...,...,...,...,...,...,...,...,...,...,...
407,1674686048193101828,เน็ต ais ไม่อั้นไม่ลดสปีด ปี 2566 #AIS เน็ตAIS #net #เน็ตaisซิมใหม่ #เน็ตaisไม่ลดสปีด #เน็ตAIS #เน็ตวันทูคอล #AIS5G #เน็ตไม่อั้นไม่ลดสปีด #เน็ตไม่อั้น #โปรเน็ตais #เน็ตฟลิกซ์ #เน็ตเอไอเอส #เน็ตมือถือ #เน็ตais #ais #Ais #เน็ตไม่ลดสปีด #วันทูคอล #AISFibre #AISPLAY #เน็ตฟลิกซ์ไทย,0,0,1,0,278,TANTAWAN014849,2023-06-30 07:47:22,2024-12-24 04:46:36,เน็ตเอไอเอส
408,1736963648411848741,มีโปรย้ายค่ายเบอร์เดิมๆคุ้มๆไหมคะ #ais #Ais #เน็ตเอไอเอส,0,0,3,0,466,kkybeiei,2023-12-19 04:16:19,2024-12-24 04:46:36,เน็ตเอไอเอส
409,1640948608521367552,หลุดดีจังนะ เวลาสำคัญๆต้องส่งงานเนี่ย #เน็ตเอไอเอส,0,0,0,0,64,Kit78065566,2023-03-29 05:26:50,2024-12-24 04:46:36,เน็ตเอไอเอส
410,1653052012693692419,จ่ายเดือนละตั้งหชายบาทเน็ตฟลิกยังดูไม่ได้ กุงึด ทุกเดือนต้องมาซื้อความเร็วเน็ตเพิ่มอีกมันใช่เรื่องไหม #เน็ตเอไอเอส,0,0,0,0,162,alittlePP15,2023-05-01 15:01:26,2024-12-24 04:46:36,เน็ตเอไอเอส


เซฟไฟล์ csv

In [31]:
save_csv(main_df)

เช็คข้อมูลเบื้องต้น

In [14]:
main_df.ID

0      1783681490318143902
1      1728921798811746633
2      1657651670560698368
3      1691777404833009925
4      1660348104267423744
              ...         
407    1674686048193101828
408    1736963648411848741
409    1640948608521367552
410    1653052012693692419
411    1664239717406367744
Name: ID, Length: 412, dtype: object

In [15]:
len(main_df['ID'].unique())

412

In [16]:
main_df.dtypes

ID                   object
Tweet                object
Replies               int64
Retweets              int64
Likes                 int64
Bookmarks             int64
Views                 int64
Username             object
PostDate     datetime64[ns]
Timestamp    datetime64[ns]
Category             object
dtype: object

In [17]:
duplicated_rows = main_df[main_df['ID'].duplicated(keep=False)]
duplicated_rows

,ID,Tweet,Replies,Retweets,Likes,Bookmarks,Views,Username,PostDate,Timestamp,Category


In [37]:
check_timestamp = main_df[['ID','Timestamp']].groupby('Timestamp').count().reset_index().sort_values(by='Timestamp')
check_timestamp

,Timestamp,ID
0,2024-11-29 01:24:19,17
1,2024-11-29 01:26:05,16
2,2024-11-29 01:26:32,5
3,2024-12-03 22:01:14,2
4,2024-12-03 22:25:48,2
5,2024-12-03 22:27:16,2
6,2024-12-03 22:27:43,3
7,2024-12-03 23:13:10,2
8,2024-12-03 23:14:32,8
9,2024-12-03 23:15:32,6


In [40]:
check_category= main_df[['ID','Category']].groupby('Category').count().reset_index()
check_category

,Category,ID
0,มหาวิทยาลัยเกษตรศาสตร์,161
1,เน็ตทรู,162
2,เน็ตเอไอเอส,89


In [25]:
check_timestamp_newest = main_df[
    (main_df['Timestamp'] == "2024-12-24 04:45:53") & 
    (main_df['Category'] == "เน็ตทรู")
][['ID', 'Timestamp', 'Tweet', 'Category']]
check_timestamp_newest

,ID,Timestamp,Tweet,Category
266,1842917915798155695,2024-12-24 04:45:53,ควรทำไงดี เน็ตคือ กว่าอัพแต่ละโพสต์ได้ โอ้ยยย ...,เน็ตทรู
267,1843886697966383536,2024-12-24 04:45:53,เน็ตบ้านลูกค้าล่มนานกว่า24ชม.แล้ว ไม่ตรวจสอบบ้...,เน็ตทรู
268,1843615244842942585,2024-12-24 04:45:53,อินเตอร์เน็ตทรูห่วยมาก #truemoveh #เน็ตทรู,เน็ตทรู
269,1864001443692421231,2024-12-24 04:45:53,#เน็ตทรู เป็นอะไร ขึ้น3G สัญญาณหายทุกๆ ครึ่งชั...,เน็ตทรู
270,1850530713994416545,2024-12-24 04:45:53,เน็ตค่ายเหี้ยนี่แม่งส้นตีนล่มทุกวันแล้วไม่แก้เ...,เน็ตทรู
...,...,...,...,...
359,1847517109565477119,2024-12-24 04:45:53,เน็ตหมุนทั้งวันทุกวันตั้งแต่6โมง ไอสัสเน็ตเหี้...,เน็ตทรู
360,1858028601393938914,2024-12-24 04:45:53,เน็ตทรูล่มรึกุมีปัญหาคนเดียว #เน็ตทรู,เน็ตทรู
361,1852988897837580646,2024-12-24 04:45:53,กูเกลียด #เน็ตทรู,เน็ตทรู
362,1863137226668048460,2024-12-24 04:45:53,สัญญาณทรูมันกระจอกจริงๆ #เน็ตทรู,เน็ตทรู


In [33]:
main_df

,ID,Tweet,Replies,Retweets,Likes,Bookmarks,Views,Username,PostDate,Timestamp,Category
0,1783681490318143902,รศ. ดร.วีระภาส คุณรัตนสิริ รองคณบดีฝ่ายสารสนเท...,0,0,0,0,296,Grad_KU,2024-04-26 02:16:21,2024-11-29 01:24:19,มหาวิทยาลัยเกษตรศาสตร์
1,1728921798811746633,มาวิ่งงาน Run For Fisheries #RunForFisheries20...,0,0,0,0,193,wissarut106,2023-11-26 23:40:53,2024-11-29 01:24:19,มหาวิทยาลัยเกษตรศาสตร์
2,1657651670560698368,ถ้าเราอยากอยู่ห้องเดียวกับเพื่อนสามารถทำยังไงไ...,1,0,0,0,450,bambysocute,2023-05-14 07:38:50,2024-11-29 01:24:19,มหาวิทยาลัยเกษตรศาสตร์
3,1691777404833009925,แอดไลน์ http://bit.ly/3AVZOt6 แอดไลน์ตอนนี้ ...,0,1,0,0,917,theplannertutor,2023-08-16 11:42:19,2024-11-29 01:24:19,มหาวิทยาลัยเกษตรศาสตร์
4,1660348104267423744,รบกวนช่วยรีหน่อยค่ะ\nเวลาประมาณ 23.40 น. ใครที...,1,38,2,0,2960,DZemb_1991,2023-05-21 18:13:30,2024-11-29 01:24:19,มหาวิทยาลัยเกษตรศาสตร์
...,...,...,...,...,...,...,...,...,...,...,...
407,1674686048193101828,เน็ต ais ไม่อั้นไม่ลดสปีด ปี 2566 #AIS เน็ตAI...,0,0,1,0,278,TANTAWAN014849,2023-06-30 07:47:22,2024-12-24 04:46:36,เน็ตเอไอเอส
408,1736963648411848741,มีโปรย้ายค่ายเบอร์เดิมๆคุ้มๆไหมคะ #ais #Ais #เ...,0,0,3,0,466,kkybeiei,2023-12-19 04:16:19,2024-12-24 04:46:36,เน็ตเอไอเอส
409,1640948608521367552,หลุดดีจังนะ เวลาสำคัญๆต้องส่งงานเนี่ย #เน็ตเอไ...,0,0,0,0,64,Kit78065566,2023-03-29 05:26:50,2024-12-24 04:46:36,เน็ตเอไอเอส
410,1653052012693692419,จ่ายเดือนละตั้งหชายบาทเน็ตฟลิกยังดูไม่ได้ กุงึ...,0,0,0,0,162,alittlePP15,2023-05-01 15:01:26,2024-12-24 04:46:36,เน็ตเอไอเอส


In [30]:
result = main_df[main_df['Category'] == "เกษตรศาสตร์"]


with pd.option_context('display.max_colwidth', None):
    display(result)

,ID,Tweet,Replies,Retweets,Likes,Bookmarks,Views,Username,PostDate,Timestamp,Category


In [22]:
print(type(main_df))

<class 'pandas.core.frame.DataFrame'>
